<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

OPENAI_API_KEY = "YOUR API KEY"

In [ ]:
import openai
import random

client = openai.OpenAI(api_key=OPENAI_API_KEY)

def generate_plans(user_query, n=5):
    """Generate multiple plans using GPT-4o-Mini."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a strategic reasoner. Given a user query, create a detailed plan to address it and then respond to the user. You will do so by using <thinking> and <response> tags. The user will only see what is in <response>, the <thinking> is just a scratchpad for you to figure out the best approach to satisfy the user's request. First, plan inside <thinking> tags, and then write your <response>."},
            {"role": "user", "content": user_query}
        ],
        n=n,
        temperature=0.7,
        max_tokens=500,
        stop=['</thinking>']
    )
    return [choice.message.content.replace('<thinking>', '') for choice in response.choices]

def evaluate_plans(plans, user_query):
    """Evaluate plans using a tournament ranking system."""
    def compare_plans(plan1, plan2):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a judge evaluating two plans. Choose the better plan based on effectiveness, feasibility, and relevance to the user's query."},
                {"role": "user", "content": f"User Query: {user_query}\n\nPlan 1: {plan1}\n\nPlan 2: {plan2}\n\nWhich plan is better? Respond with either '1' or '2'."}
            ],
            temperature=0.2,
            max_tokens=10
        )
        return 1 if response.choices[0].message.content.strip() == "1" else 2

    # Tournament ranking — to improve, run twice with swapped positions for each plan combo for greater precision, and parallelize for speed
    winners = plans
    while len(winners) > 1:
        next_round = []
        for i in range(0, len(winners), 2):
            if i + 1 < len(winners):
                winner = winners[i] if compare_plans(winners[i], winners[i+1]) == 1 else winners[i+1]
                next_round.append(winner)
            else:
                next_round.append(winners[i])
        winners = next_round

    return winners[0]

def generate_response(best_plan, user_query):
    """Generate the final response based on the best plan."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the given plan to create a detailed and high-quality response to the user's query."},
            {"role": "user", "content": f"User Query: {user_query}\n\nPlan: {best_plan}\n\nGenerate a detailed response based on this plan."}
        ],
        temperature=0.5,
        max_tokens=2000
    )
    return response.choices[0].message.content

def improved_ai_output(user_query, num_plans=20):
    """Main function to improve AI output quality."""
    print("Generating plans...")
    plans = generate_plans(user_query, n=num_plans)

    print("Evaluating plans...")
    best_plan = evaluate_plans(plans, user_query)

    print("Generating final response...")
    final_response = generate_response(best_plan, user_query)

    return {
        "user_query": user_query,
        "best_plan": best_plan,
        "final_response": final_response
    }

if __name__ == "__main__":
    user_query = "How do I get a computer out of a bootloop?"
    result = improved_ai_output(user_query)

    print("\nUser Query:", result["user_query"])
    print("\nBest Plan:", result["best_plan"])
    print("\nFinal Response:", result["final_response"])